# Generate the systems for multistates


In [1]:
#IMPORT
import os, sys, glob
import time

from pygromos.gromos.gromosPP import GromosPP
from pygromos.gromos.gromosXX import GromosXX

from pygromos.files.coord import cnf
from pygromos.files import imd
from pygromos.files.blocks import imd_blocks

from pygromos.data import imd_templates
from pygromos.data import solvents

import restraintmaker

#CHANGE HERE
gromosPP_bin_path = "/home/bschroed/Documents/code/gromosPP/installed/bin"
gromosXX_bin_path ="/home/bschroed/Documents/code/gromos_2019_EDSemin/installed/bin"
restraintmaker_path = os.path.abspath(os.path.dirname(restraintmaker.__file__)+"/..")



/home/bschroed/Documents/code/restraintmaker/restraintmaker/submodules/pygromos/pygromos/files/topology/top.py:20: UserWarning: Module topo-files is under Development and not entirely implemented!
  warnings.warn("Module topo-files is under Development and not entirely implemented!")


## Path definitions
generate the subfolders for the systems - no changes required here.

In [2]:
sets_dir = restraintmaker_path+"/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets"
resn_lib_path = sets_dir+"/resn_lib.lib"
mstate_dir = sets_dir+"/multistate"


atb_dirs = restraintmaker_path+"/devtools/otherScripts/b_ATB_solvationFreeEnergies/ATB_molecules"
sys.path.append(atb_dirs+"/..")

import utils_test_set_ATB as utils

if(not os.path.exists(mstate_dir)):
    os.mkdir(mstate_dir)

##get all_single file_tops:
all_tops = glob.glob(atb_dirs+"/*/*top")
state_all_tops={os.path.basename(value).split(".")[0]: value for value in all_tops}
all_mstate_sys = list(filter(os.path.isdir, glob.glob(mstate_dir+"/*")))

emin_imd = imd_templates.template_emin
gromXX = GromosXX(gromosXX_bin_path)
gromPP = GromosPP(gromosPP_bin_path)



GROMOSPATH /home/bschroed/Documents/code/gromos_2019_EDSemin/installed/bin/


## Build gromos files

In [4]:
#Systems
##multi_state different_sets
print("system name\t states")
for name,state_set in utils.multistate_ligand_sets.items():
    print(name, state_set)
    system_tops = [state_all_tops[x] for x in state_set]

    #build eds system
    print(all_mstate_sys)
    out_dir = list(filter(lambda x: name+"_" in x, all_mstate_sys))[0]
    out_prefix_path = out_dir+"/"+os.path.basename(out_dir)
    out_top_path, out_ptp_path = gromPP.prep_eds(in_top_paths=system_tops, number_of_eds_states=len(system_tops), out_file_path=out_prefix_path)

    #generate cnf
    in_pdb = out_prefix_path+".pdb"
    out_cnf_path = gromPP.pdb2gromos(in_pdb_path=in_pdb, in_top_path=out_top_path, in_lib_path=resn_lib_path,
                                     out_cnf_path=out_prefix_path+".cnf", )
    ##build box
    from pygromos.files.blocks.coord_blocks import GENBOX, Pbc
    box_cnf = cnf.Cnf(out_cnf_path)
    box_cnf.supress_atomPosition_singulrarities()
    box_cnf.GENBOX = GENBOX(pbc=Pbc.rectangular,
                            length=[3.610220118 for x in range(3)],
                            angles=[90 for x in range(3)],
                            euler=[0 for x in range(3)],
                            origin=[0 for x in range(3)])
    box_cnf.write(out_cnf_path)

    time.sleep(3)
    gromPP.frameout(in_top_path=out_top_path, in_coord_path=out_cnf_path, periodic_boundary_condition="v",
                    out_file_path=out_prefix_path+"_box.pdb", out_file_format="pdb", time=0, verbose=True)


    # SOLVATE
    out_solv_cnf_path=out_prefix_path+"_spc.cnf"
    gromPP.sim_box(in_top_path=out_top_path, minwall=None, in_cnf_path=out_cnf_path, in_solvent_cnf_file_path=solvents.spc,
                   out_cnf_path=out_solv_cnf_path, periodic_boundary_condition=None, gathering_method=None,  boxsize=True)
    time.sleep(2)

    gromPP.frameout(in_top_path=out_top_path, in_coord_path=out_solv_cnf_path, periodic_boundary_condition="r cog",
                    out_file_path=out_prefix_path+"_solv.pdb", include="all", out_file_format="pdb", time=0)


    # Emin solv

    emin_dir = out_dir+"/solv_emin"
    emin_outprefix = emin_dir+"/"+os.path.basename(out_dir)+"_emin"

    if(not os.path.exists(emin_dir)):
        os.mkdir(emin_dir)

    cnf_file = cnf.Cnf(out_solv_cnf_path)
    residues = cnf_file.get_residues()
    print(residues)
    import numpy as np
    t = list(map(lambda x: np.sum(list(x.values())) if(type(x) is dict) else x, residues.values()))
    print(t)
    all_atoms = np.sum(t)
    del residues["SOLV"]
    all_lig_atoms =np.sum(list(map(lambda x: np.sum(list(x.values())), residues.values())))
    in_por_path,_ = cnf_file.write_possrespec( out_path=emin_outprefix+".por", residues=list(residues.keys()))
    in_rpf_path,_ = cnf_file.write_refpos(emin_outprefix+".rpf")

    in_imd_path = emin_outprefix+".imd"
    imd_file = imd.Imd(emin_imd)

    imd_file.SYSTEM.NSM = (all_atoms - all_lig_atoms)//3

    imd_file.EDS = imd_blocks.EDS(NUMSTATES=len(state_set),S=1, EIR=[0 for x in range(len(state_set))])

    multibath_block = imd_blocks.MULTIBATH(ALGORITHM=1, NBATHS=2, TEMP0=[298,298],
                                           TAU=[0.1,0.1], DOFSET=2,
                                           LAST=[all_lig_atoms, all_atoms], COMBATH=[1,2], IRBATH=[1,2] )

    #imd_file.add_block(multibath_block.name, multibath_block)
    imd_file.MULTIBATH = multibath_block
    imd_file.write(in_imd_path)

    gromXX.md_run(in_topo_path=out_top_path, in_imd_path=in_imd_path, in_coord_path=out_solv_cnf_path,
                  out_prefix=emin_outprefix, in_pert_topo_path=out_ptp_path,
                  in_refpos_path=in_rpf_path, in_posresspec_path=in_por_path)


system name	 states
all ['_O6T', '_O71', 'G277', 'S002', '8018', 'M030', '6KET', 'F313', '_P8I', 'M097', 'G078', 'M218', '6J29', 'G209', 'E1VB', 'TVVS', '_O70']
['/home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/multistate/flat_10', '/home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/multistate/challenging_5', '/home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/multistate/singles_10', '/home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/multistate/all_17', '/home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/multistate/easy_6']
gromosPP.frameout: command:
/home/bschroed/Documents/code/gromosPP/installed/bin/frameout @topo /home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/multistate/all_17/all_17.